In [ ]:
import datetime
import json
import os
import sys
import time
from configparser import ConfigParser
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split

sys.path.append('../')
from models.metrics import dice_loss, hamming_score
from models.pnsamp_2d import PNSAMP_2D
from utils.directory import check_or_create
from utils.data_generators import DicomDataGenerator

# Training

In [ ]:
parser = ConfigParser()
parser.read('../project.conf')

# Get Directory setting
# add '../' since you are in the a notebook
saved_weights_path = check_or_create('../' + parser.get('train', 'SAVED_WEIGHTS_PATH'))
checkpoints_path = check_or_create('../' + parser.get('train', 'CHECKPOINTS_PATH'))
history_path = check_or_create('../' + parser.get('train', 'HISTORY_PATH'))
data_path = check_or_create('../' + parser.get('train', 'DATA_PATH'))
batch_size = int(parser.get('train', 'BATCH_SIZE'))
image_size = int(parser.get('train', 'IMAGE_SIZE'))
variant = parser.get('train', 'VARIANT')
epochs = int(parser.get('train', 'EPOCHS'))
test_ratio = float(parser.get('train', 'TEST_RATIO'))

In [ ]:
df = pd.read_csv(os.path.join(data_path, 'meta/meta_info.csv'),
                         dtype={'patient_id': str,
                                'nodule_no': str,
                                'slice_no': str})

# use only non-clean scans (scans that contains at least one nodule) for training
df = df[df['is_clean'] == False]

def get_paths(x):
    patient_img_path = os.path.join(data_path, 'image', 'LIDC-IDRI-' + x[0])
    patient_mask_path = os.path.join(data_path, 'mask', 'LIDC-IDRI-' + x[0])
    return [os.path.join(patient_img_path, x[1] + '.npy'), os.path.join(patient_mask_path, x[2] + '.npy')]

temp = df[['patient_id',
                'original_image',
                'mask_image']].values

paths = list(map(get_paths, temp))
df_paths = pd.DataFrame(paths, columns=['img_path', 'mask_path'])

df.reset_index(drop=True, inplace=True)
df_paths.reset_index(drop=True, inplace=True)

df = pd.concat([df, df_paths], axis=1, sort=False)

df.head()

In [ ]:
features = ['subtlety',
            'margin',
            'lobulation',
            'texture',]

"""from sklearn.preprocessing import MinMaxScaler
df[features] = MinMaxScaler().fit_transform(df[features])
df = df.sample(frac=1).reset_index(drop=True)
df.head()"""

In [ ]:
num_batches = int(len(df) / batch_size)

print("|=============BUILDING GENERATOR=============|\n")
datagen = DicomDataGenerator(df,
                             img_path_col_name='img_path',
                             mask_path_col_name='mask_path',
                             features_cols=features,
                             batch_size=batch_size,
                             target_size=(image_size, image_size, 1))

traingen, valigen = train_test_split(datagen, test_size=test_ratio)

In [ ]:
model = PNSAMP_2D(num_attributes=len(features), input_size=(image_size, image_size, 1), variant=variant)
model.summary()

# Instantiate an optimizer.
optimizer = tf.keras.optimizers.Adam()

# Instantiate a loss function.
# loss_fn1 = tf.keras.losses.BinaryCrossentropy()
loss_fn1 = dice_loss
loss_fn2 = tf.keras.losses.SparseCategoricalCrossentropy()
loss_fn3 = tf.keras.losses.SparseCategoricalCrossentropy()

# setup training checkpoints
checkpoint = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, net=model)
manager = tf.train.CheckpointManager(
            checkpoint,
            os.path.join(checkpoints_path, variant),
            max_to_keep=3
        )

checkpoint.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
    print("Note: Starting training with model restored from {}".format(manager.latest_checkpoint))
else:
    print("Note: Starting training from scratch.")

In [ ]:
from sklearn.metrics import accuracy_score

# record of training
history = {
            'train_loss': [],
            'val_loss': [],
            'training_time': [],
    'train_acc': [],
            'train_ham': [],
            'val_acc': [],
            'val_ham': [],
        }

# the validation will be use save a
# checkpoint of the model for the
# best loss
best_validation_loss = np.inf

print("|==================TRAINING==================|\n")
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))

    total_loss1 = 0.0
    total_loss2 = 0.0
    total_loss3 = 0.0
    total_acc_mal = 0.0
    total_hamming = 0.0
    start_train_time = time.time()
    i = 0
    # Iterate over the batches of the train dataset.
    for _, (x_batch_train, mask_batch_train, feats_batch_train, mal_batch_train) in enumerate(traingen):
        i += 1
        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape(persistent=True) as tape:
            # Run the forward pass of the layer. The operations that the layer applies to its inputs are
            # going to be recorded on the GradientTape. segmentation_logits, multi_regr_logits, class_logits
            segmentation_logits, multi_regr_logits, class_logits = model(x_batch_train, training=True)
            
            # Logits for this minibatch
            # Compute the loss value for this minibatch.
            loss_value1 = loss_fn1(mask_batch_train, segmentation_logits)
            loss_value2 = loss_fn2(feats_batch_train, multi_regr_logits)
            loss_value3 = loss_fn3(mal_batch_train, class_logits)
            print(loss_value1.numpy(), loss_value2.numpy(), loss_value3.numpy())
            
            
            # add different losses to total loss
            total_loss1 += loss_value1.numpy()
            total_loss2 += loss_value1.numpy()
            total_loss3 += loss_value1.numpy()
            total_acc_mal += accuracy_score(
                mal_batch_train.numpy(),
                tf.math.argmax(class_logits, axis=-1).numpy())
            total_hamming += hamming_score(
                feats_batch_train.numpy(),
                tf.math.argmax(multi_regr_logits, axis=-1).numpy())

        """MULTI-CLASSIFICATOIN"""
        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        # grads = tape.gradient(loss_value1, model.trainable_weights)
        grads = tape.gradient([loss_value1, loss_value2, loss_value3], model.trainable_weights)

        optimizer.apply_gradients(
            (grad, var) 
            for (grad, var) in zip(grads, model.trainable_variables) 
            if grad is not None
        )

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        # optimizer.apply_gradients(zip(grads, model.trainable_weights))

    end_train_time = time.time()
    history['training_time'].append(end_train_time - start_train_time)
    history['train_loss'].append(total_loss1 / i)
    history['train_acc'].append(total_acc_mal / i)
    history['train_ham'].append(total_hamming / i)

    # Iterate over the batches of the dataset.
    total_loss1 = 0.0
    total_loss2 = 0.0
    total_loss3 = 0.0
    total_acc_mal = 0.0
    total_hamming = 0.0
    start_train_time = time.time()
    i = 0
    for _, (x_batch_val, mask_batch_val, feats_batch_val, mal_batch_val) in enumerate(valigen):
        i += 1
        with tf.GradientTape(persistent=True) as tape:
            segmentation_logits, multi_regr_logits, class_logits = model(x_batch_val, training=True)

            # Compute the loss value for this mini batch.
            loss_value1 = loss_fn1(mask_batch_val, segmentation_logits)
            loss_value2 = loss_fn2(feats_batch_val, multi_regr_logits)
            loss_value3 = loss_fn3(mal_batch_val, class_logits)
            print(loss_value1.numpy(), loss_value2.numpy(), loss_value3.numpy())

        # add different losses to total loss
        total_loss1 += loss_value1.numpy()
        total_loss2 += loss_value1.numpy()
        total_loss3 += loss_value1.numpy()
        total_acc_mal += accuracy_score(
            mal_batch_val.numpy(),
            tf.math.argmax(class_logits, axis=-1).numpy())
        total_hamming += hamming_score(
            feats_batch_train.numpy(),
            tf.math.argmax(multi_regr_logits, axis=-1).numpy())

    history['val_loss'].append(total_loss1 / i)
    history['val_acc'].append(total_acc_mal / i)
    history['val_ham'].append(total_hamming / i)

    # save model if performance is better (loss is lower)
    if history['val_loss'][-1] < best_validation_loss:
        save_path = manager.save()
        print("Saved checkpoint for step {}: {}, loss {:1.3f}".format(int(checkpoint.step),
                                                                              save_path,
                                                                              history['val_loss'][-1]))
        model.save_weights(os.path.join(saved_weights_path, variant), save_format='tf')
        checkpoint.step.assign_add(1)
        best_validation_loss = history['val_loss'][-1]

In [ ]:
print('Final training loss', history['train_loss'][-1])
print('Final validation loss', history['val_loss'][-1])

# save history file
today = datetime.datetime.now()

if today.hour < 12:
    h = "00"
else:
    h = "12"
    
file_path = check_or_create(os.path.join(history_path, variant))

with open(os.path.join(file_path, 'history_{}.json'.format(today.strftime('%Y%m%d') + h +
                                                                                 str(today.minute))),'w') as fp:
    json.dump(history, fp)

In [ ]:
history